In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

pickle_file = '../lesson1/notMNIST_cor1.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (15000, 28, 28) (15000,)
Test set (16355, 28, 28) (16355,)


In [2]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (15000, 784) (15000, 10)
Test set (16355, 784) (16355, 10)


In [3]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [61]:
batch_size = 128
hidden_units = 1024
lambd = 0.001


graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_units]))
  biases1 = tf.Variable(tf.zeros([hidden_units]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_units, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
    
  def forward_propagation(tf_dataset, *args):
      fist_level = tf.nn.relu(tf.add(tf.matmul(tf_dataset, args[0]), args[1]))
      out = tf.add(tf.matmul(fist_level, args[2]), args[3])
      return out
  
  logits = forward_propagation(tf_train_dataset, weights1, biases1, weights2, biases2)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
  
  # Loss function using L2 Regularization
  regularizer = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)
  loss = tf.reduce_mean(loss + lambd * regularizer)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_propagation(tf_valid_dataset, weights1, biases1, weights2, biases2))
  test_prediction = tf.nn.softmax(forward_propagation(tf_test_dataset, weights1, biases1, weights2, biases2))

In [62]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 738.390747
Minibatch accuracy: 5.5%
Validation accuracy: 29.3%
Minibatch loss at step 500: 195.189896
Minibatch accuracy: 82.8%
Validation accuracy: 79.3%
Minibatch loss at step 1000: 114.704529
Minibatch accuracy: 81.2%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 68.339058
Minibatch accuracy: 82.8%
Validation accuracy: 83.4%
Minibatch loss at step 2000: 41.334461
Minibatch accuracy: 86.7%
Validation accuracy: 85.0%
Minibatch loss at step 2500: 25.228209
Minibatch accuracy: 83.6%
Validation accuracy: 86.0%
Minibatch loss at step 3000: 15.681702
Minibatch accuracy: 84.4%
Validation accuracy: 86.2%
Test accuracy: 86.1%


In [42]:
# Problem overfitting

train_dataset_2 = train_dataset[:800, :]
train_labels_2 = train_labels[:800]

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_2.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_2[offset:(offset + batch_size), :]
    batch_labels = train_labels_2[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 707.291260
Minibatch accuracy: 12.5%
Validation accuracy: 22.9%
Minibatch loss at step 500: 191.002731
Minibatch accuracy: 100.0%
Validation accuracy: 75.6%
Minibatch loss at step 1000: 115.863953
Minibatch accuracy: 100.0%
Validation accuracy: 75.5%
Minibatch loss at step 1500: 70.274857
Minibatch accuracy: 100.0%
Validation accuracy: 76.0%
Minibatch loss at step 2000: 42.620617
Minibatch accuracy: 100.0%
Validation accuracy: 75.8%
Minibatch loss at step 2500: 25.848894
Minibatch accuracy: 100.0%
Validation accuracy: 76.3%
Minibatch loss at step 3000: 15.678967
Minibatch accuracy: 100.0%
Validation accuracy: 76.6%
Test accuracy: 76.1%


In [113]:
batch_size = 128
hidden_units = 1024
lambd = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_units]))
    biases1 = tf.Variable(tf.zeros([hidden_units]))
    weights2 = tf.Variable(
    tf.truncated_normal([hidden_units, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    keep_prob = tf.placeholder(tf.float32)
  

  # Training computation.
    
    def forward_propagation(tf_dataset, keep_prob, *args):
        fist_level = tf.nn.relu(tf.add(tf.matmul(tf_dataset, args[0]), args[1]))
        drop_out = tf.nn.dropout(fist_level, keep_prob)
        out = tf.add(tf.matmul(drop_out, args[2]), args[3])
        return out

    logits = forward_propagation(tf_train_dataset, keep_prob, weights1, biases1, weights2, biases2)

    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))

    # Loss function using L2 Regularization
    regularizer = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)
    loss = tf.reduce_mean(loss + lambd * regularizer)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(forward_propagation(tf_valid_dataset, 1, weights1, biases1, weights2, biases2))
    test_prediction = tf.nn.softmax(forward_propagation(tf_test_dataset, 1, weights1, biases1, weights2, biases2))

In [114]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.7}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 674.245422
Minibatch accuracy: 13.3%
Validation accuracy: 18.3%
Minibatch loss at step 500: 201.003510
Minibatch accuracy: 77.3%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 117.012962
Minibatch accuracy: 75.8%
Validation accuracy: 80.4%
Minibatch loss at step 1500: 68.555405
Minibatch accuracy: 79.7%
Validation accuracy: 82.4%
Minibatch loss at step 2000: 41.472336
Minibatch accuracy: 82.0%
Validation accuracy: 84.2%
Minibatch loss at step 2500: 25.298134
Minibatch accuracy: 80.5%
Validation accuracy: 85.9%
Minibatch loss at step 3000: 15.735685
Minibatch accuracy: 79.7%
Validation accuracy: 86.1%
Test accuracy: 85.7%


In [158]:
# many_layers

batch_size = 128
hidden_units_1 = 1024
hidden_units_2 = 512
hidden_units_3 = 128
# hidden_units_4 = 256
# hidden_units_5 = 256
# hidden_units_6 = 128
lambd = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_units_1]))
    biases1 = tf.Variable(tf.zeros([hidden_units_1]))
    
    weights2 = tf.Variable(tf.truncated_normal([hidden_units_1, hidden_units_2]))
    biases2 = tf.Variable(tf.zeros([hidden_units_2]))
    
    weights3 = tf.Variable(
    tf.truncated_normal([hidden_units_2, hidden_units_3]))
    biases3 = tf.Variable(tf.zeros([hidden_units_3]))
    
    weights4 = tf.Variable(tf.truncated_normal([hidden_units_3, num_labels]))
    biases4 = tf.Variable(tf.zeros([num_labels]))
    
#     weights5 = tf.Variable(
#     tf.truncated_normal([hidden_units_4, hidden_units_5]))
#     biases5 = tf.Variable(tf.zeros([hidden_units_5]))
    
#     weights6 = tf.Variable(tf.truncated_normal([hidden_units_5, hidden_units_6]))
#     biases6 = tf.Variable(tf.zeros([hidden_units_6]))
    
#     weights7 = tf.Variable(tf.truncated_normal([hidden_units_6, num_labels]))
#     biases7 = tf.Variable(tf.zeros([num_labels]))
    
    keep_prob = tf.placeholder(tf.float32)
    
      
    parametrs = {'weights1': weights1,
                 'weights2': weights2, 
                 'weights3': weights3,
                 'weights4': weights4,
                 'biases1': biases1,
                 'biases2': biases2,
                 'biases3': biases3,
                 'biases4': biases4}

    # Training computation.

    def forward_propagation(tf_dataset, keep_prob, parametrs):
      fist_level = tf.nn.relu(tf.add(tf.matmul(tf_dataset, parametrs['weights1']), parametrs['biases1']))
      drop_out = tf.nn.dropout(fist_level, keep_prob)
      second_level = tf.nn.relu(tf.add(tf.matmul(drop_out, parametrs['weights2']), parametrs['biases2']))
      drop_out = tf.nn.dropout(second_level, keep_prob)
      third_level = tf.nn.sigmoid(tf.add(tf.matmul(drop_out, parametrs['weights3']), parametrs['biases3']))
      drop_out = tf.nn.dropout(third_level, keep_prob)
#       fourth_level = tf.nn.sigmoid(tf.add(tf.matmul(drop_out, parametrs['weights4']), parametrs['biases4']))
#       drop_out = tf.nn.dropout(fourth_level, keep_prob)
#       fifth_level = tf.nn.sigmoid(tf.add(tf.matmul(drop_out, parametrs['weights5']), parametrs['biases5']))
#       drop_out = tf.nn.dropout(fifth_level, keep_prob)
#       six_level = tf.nn.sigmoid(tf.add(tf.matmul(drop_out, parametrs['weights6']), parametrs['biases6']))
      out = tf.add(tf.matmul(drop_out, parametrs['weights4']), parametrs['biases4'])
      return out

    logits = forward_propagation(tf_train_dataset, keep_prob, parametrs)

    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))

    # Loss function using L2 Regularization
    regularizer = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3) + tf.nn.l2_loss(weights4)
    
    loss = tf.reduce_mean(loss + lambd * regularizer)

    # Optimizer.
    # Decaying learning rate
    global_step = tf.Variable(0)  # count the number of steps taken.
    start_learning_rate = 0.6
    learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 5000, 0.95)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

#     optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

#     optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(forward_propagation(tf_valid_dataset, 1, parametrs))
    test_prediction = tf.nn.softmax(forward_propagation(tf_test_dataset, 1, parametrs))

In [159]:
num_steps = 50001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.7}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 551.557617
Minibatch accuracy: 18.8%
Validation accuracy: 9.9%
Minibatch loss at step 500: 308.161774
Minibatch accuracy: 14.1%
Validation accuracy: 10.8%
Minibatch loss at step 1000: 170.765167
Minibatch accuracy: 14.1%
Validation accuracy: 23.1%
Minibatch loss at step 1500: 95.032204
Minibatch accuracy: 34.4%
Validation accuracy: 36.0%
Minibatch loss at step 2000: 53.035732
Minibatch accuracy: 56.2%
Validation accuracy: 67.1%
Minibatch loss at step 2500: 30.114336
Minibatch accuracy: 66.4%
Validation accuracy: 77.6%
Minibatch loss at step 3000: 17.386379
Minibatch accuracy: 71.9%
Validation accuracy: 81.6%
Minibatch loss at step 3500: 9.791650
Minibatch accuracy: 79.7%
Validation accuracy: 83.5%
Minibatch loss at step 4000: 6.027232
Minibatch accuracy: 80.5%
Validation accuracy: 84.5%
Minibatch loss at step 4500: 3.611290
Minibatch accuracy: 80.5%
Validation accuracy: 85.9%
Minibatch loss at step 5000: 2.274485
Minibatch accuracy: 85.2%
Validatio

Validation accuracy: 89.0%
Minibatch loss at step 45000: 0.609919
Minibatch accuracy: 88.3%
Validation accuracy: 89.4%
Minibatch loss at step 45500: 0.558379
Minibatch accuracy: 86.7%
Validation accuracy: 89.2%
Minibatch loss at step 46000: 0.481403
Minibatch accuracy: 90.6%
Validation accuracy: 89.0%
Minibatch loss at step 46500: 0.581514
Minibatch accuracy: 87.5%
Validation accuracy: 89.1%
Minibatch loss at step 47000: 0.686891
Minibatch accuracy: 86.7%
Validation accuracy: 89.2%
Minibatch loss at step 47500: 0.558065
Minibatch accuracy: 87.5%
Validation accuracy: 89.4%
Minibatch loss at step 48000: 0.428256
Minibatch accuracy: 91.4%
Validation accuracy: 89.4%
Minibatch loss at step 48500: 0.486155
Minibatch accuracy: 89.8%
Validation accuracy: 89.3%
Minibatch loss at step 49000: 0.536376
Minibatch accuracy: 88.3%
Validation accuracy: 88.9%
Minibatch loss at step 49500: 0.647348
Minibatch accuracy: 85.9%
Validation accuracy: 89.1%
Minibatch loss at step 50000: 0.698362
Minibatch accu

In [153]:
train_dataset.shape

(200000, 784)